In [1]:
import numpy as np
import pandas as pd
import regex as re
import contractions
import string

In [2]:
notes = pd.read_csv("../mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/discharge.csv.gz", compression="gzip")

In [3]:
notes = notes.drop(columns= ['note_type'])
notes = notes.drop(columns= ['charttime'])
notes = notes.drop(columns= ['storetime'])
notes = notes.drop(columns= ['note_id'])
notes = notes.drop(columns= ['note_seq'])

https://www.analyticsvidhya.com/blog/2021/06/must-known-techniques-for-text-preprocessing-in-nlp/

Remove newlines

In [4]:
notes['text'] = notes['text'].replace('\n','', regex=True)

Expand contractions

In [17]:
notes['text'] = notes['text'].apply(lambda txt:contractions.fix(txt))

Lowercase the text

In [5]:
notes['text'] = notes['text'].apply(lambda txt: txt.lower())

In [12]:
notes['text'] = notes['text'].replace(' name: ___ unit no: ___ admission date: ___ discharge date: ___ date of birth: ___ ','',regex=True)
notes['text'] = notes['text'].replace('followup instructions:___','',regex=True)

In [20]:
notes['text'] = notes['text'].replace(' name: ___. unit no: ___ admission date: ___ discharge date: ___ date of birth: ___ ','',regex=True)

In [51]:
notes['text'] = notes['text'].replace(' name: ___ ___ no: ___ admission date: ___ discharge date: ___ date of birth: ___ ','',regex=True)

remove_digits

In [7]:
notes['text'] = notes['text'].str.replace("(\s\d+)+\s", " ", regex=True)

In [8]:
notes['text'] = notes['text'].str.replace("[^A-Za-z0-9]+", " ", regex=True)

In [9]:
notes['text'] = notes['text'].str.replace("\s+", " ", regex=True)

Remove punctuations

In [114]:
notes['text'] = notes['text'].apply(lambda txt: re.sub('[%s]' % re.escape('_='),'',txt))

In [168]:
mypunc = string.punctuation
mypunc = mypunc.replace('.','')
mypunc = mypunc.replace('/','')
mypunc = mypunc.replace(':','')
mypunc

'!"#$%&\'()*+,-;<=>?@[\\]^_`{|}~'

In [33]:
notes['text'] = notes['text'].apply(lambda txt: re.sub('[%s]' % re.escape(mypunc),'',txt))

Remove stopwords(not good)

In [34]:
from nltk.corpus import stopwords
mystpwrd = stopwords.words('english')
mystpwrd.remove('not')
notes['text'] = notes['text'].apply(lambda txt: " ".join([word for word in str(txt).split() if word not in mystpwrd]))

Rephrase

Lemmatization

In [35]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
notes["text"] = notes["text"].apply(lambda text: lemmatize_words(text))

Remove extra Spaces

In [9]:
notes['text'] = notes['text'].apply(lambda txt: re.sub(' +',' ',txt))

In [121]:
notes.to_csv('lower_=removed_notes.csv', index=False)

Importing Icd and extract icd_10 and 10 top icd codes

In [11]:
icd = pd.read_csv("../diagnoses_icd.csv.gz", compression="gzip")

In [ ]:
icd_10 = icd[icd["icd_version"]==10]
top_10_code = icd_10["icd_code"].value_counts().head(10)
top_10_icd = icd_10.loc[icd_10['icd_code'].isin(top_10_code.index)].reset_index(drop=True)

In [13]:
top_10_icd_gpb = top_10_icd.groupby("hadm_id")["icd_code"].agg(list).reset_index()

In [14]:
for i in top_10_code.index:
    top_10_icd_gpb[i] = top_10_icd_gpb['icd_code'].apply(lambda x: 1 if i in x else 0)

In [16]:
top_10_icd_gpb.to_csv('ICD_oneHoted.csv', index=False)

In [17]:
icd_10_notes = top_10_icd_gpb.merge(notes,on = ["hadm_id"])

In [19]:
icd_10_notes.to_csv('Preprocessed_datav2.csv', index=False)